In [29]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:
!nvidia-smi

Mon Feb 10 20:46:08 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.48.02    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   67C    P0    31W /  70W |   2463MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [31]:
import torch
torch.__version__

'1.4.0'

In [32]:
bart = torch.hub.load('pytorch/fairseq', 'bart.large')
bart.eval()  # disable dropout (or leave in train mode to finetune)

Using cache found in /root/.cache/torch/hub/pytorch_fairseq_master


BARTHubInterface(
  (model): BARTModel(
    (encoder): TransformerEncoder(
      (embed_tokens): Embedding(50265, 1024, padding_idx=1)
      (embed_positions): LearnedPositionalEmbedding(1026, 1024, padding_idx=1)
      (layers): ModuleList(
        (0): TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        )
        (1): TransformerEnc

In [0]:
for p in bart.parameters():
  p.requires_grad = False

## Convert whole dataset to bart embeddings

In [125]:
import pandas as pd

PATH = '/content/drive/My Drive/Kaggle/Google QUEST Q&A Labeling/data/base/'
OUTPUT_PATH = '/content/drive/My Drive/Kaggle/Google QUEST Q&A Labeling/output/2/'

df_train = pd.read_csv(PATH+'train.csv')
df_test = pd.read_csv(PATH+'test.csv')
df_sub = pd.read_csv(PATH+'sample_submission.csv')
print('train shape =', df_train.shape)
print('test shape =', df_test.shape)

output_categories = list(df_train.columns[11:])
input_categories = list(df_train.columns[[1,2,5]])
print('\noutput categories:\n\t', output_categories)
print('\ninput categories:\n\t', input_categories)

train shape = (6079, 41)
test shape = (476, 11)

output categories:
	 ['question_asker_intent_understanding', 'question_body_critical', 'question_conversational', 'question_expect_short_answer', 'question_fact_seeking', 'question_has_commonly_accepted_answer', 'question_interestingness_others', 'question_interestingness_self', 'question_multi_intent', 'question_not_really_a_question', 'question_opinion_seeking', 'question_type_choice', 'question_type_compare', 'question_type_consequence', 'question_type_definition', 'question_type_entity', 'question_type_instructions', 'question_type_procedure', 'question_type_reason_explanation', 'question_type_spelling', 'question_well_written', 'answer_helpful', 'answer_level_of_information', 'answer_plausible', 'answer_relevance', 'answer_satisfaction', 'answer_type_instructions', 'answer_type_procedure', 'answer_type_reason_explanation', 'answer_well_written']

input categories:
	 ['question_title', 'question_body', 'answer']


In [35]:
df_train.head()

,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,question_not_really_a_question,question_opinion_seeking,question_type_choice,question_type_compare,question_type_consequence,question_type_definition,question_type_entity,question_type_instructions,question_type_procedure,question_type_reason_explanation,question_type_spelling,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,0,What am I losing when using extension tubes in...,After playing around with macro photography on...,ysap,https://photo.stackexchange.com/users/1024,"I just got extension tubes, so here's the skin...",rfusca,https://photo.stackexchange.com/users/1917,http://photo.stackexchange.com/questions/9169/...,LIFE_ARTS,photo.stackexchange.com,1.000000,0.333333,0.000000,0.0,0.0,0.0,1.000000,1.000000,0.000000,0.0,1.000000,0.000000,0.000000,0.0,0.000000,0.0,1.0,0.000000,0.000000,0.0,1.000000,1.000000,0.666667,1.000000,1.000000,0.800000,1.0,0.000000,0.000000,1.000000
1,1,What is the distinction between a city and a s...,I am trying to understand what kinds of places...,russellpierce,https://rpg.stackexchange.com/users/8774,It might be helpful to look into the definitio...,Erik Schmidt,https://rpg.stackexchange.com/users/1871,http://rpg.stackexchange.com/questions/47820/w...,CULTURE,rpg.stackexchange.com,1.000000,1.000000,0.000000,0.5,1.0,1.0,0.444444,0.444444,0.666667,0.0,0.000000,0.666667,0.666667,0.0,0.333333,0.0,0.0,0.000000,0.333333,0.0,0.888889,0.888889,0.555556,0.888889,0.888889,0.666667,0.0,0.000000,0.666667,0.888889
2,2,Maximum protusion length for through-hole comp...,I'm working on a PCB that has through-hole com...,Joe Baker,https://electronics.stackexchange.com/users/10157,Do you even need grooves? We make several pro...,Dwayne Reid,https://electronics.stackexchange.com/users/64754,http://electronics.stackexchange.com/questions...,SCIENCE,electronics.stackexchange.com,0.888889,0.666667,0.000000,1.0,1.0,1.0,0.666667,0.444444,0.333333,0.0,0.333333,0.000000,0.000000,0.0,0.000000,0.0,1.0,0.333333,0.333333,0.0,0.777778,0.777778,0.555556,1.000000,1.000000,0.666667,0.0,0.333333,1.000000,0.888889
3,3,Can an affidavit be used in Beit Din?,"An affidavit, from what i understand, is basic...",Scimonster,https://judaism.stackexchange.com/users/5151,"Sending an ""affidavit"" it is a dispute between...",Y e z,https://judaism.stackexchange.com/users/4794,http://judaism.stackexchange.com/questions/551...,CULTURE,judaism.stackexchange.com,0.888889,0.666667,0.666667,1.0,1.0,1.0,0.444444,0.444444,0.000000,0.0,0.000000,1.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.888889,0.833333,0.333333,0.833333,1.000000,0.800000,0.0,0.000000,1.000000,1.000000
4,5,How do you make a binary image in Photoshop?,I am trying to make a binary image. I want mor...,leigero,https://graphicdesign.stackexchange.com/users/...,Check out Image Trace in Adobe Illustrator. \n...,q2ra,https://graphicdesign.stackexchange.com/users/...,http://graphicdesign.stackexchange.com/questio...,LIFE_ARTS,graphicdesign.stackexchange.com,1.000000,0.666667,0.000000,1.0,1.0,1.0,0.666667,0.666667,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,1.0,0.000000,1.000000,0.0,1.000000,1.000000,0.666667,1.000000,1.000000,0.800000,1.0,0.000000,1.000000,1.000000


In [0]:
# torch.cuda.empty_cache()

In [0]:
TRAIN_BATCH_SIZE = 8
EVAL_BATCH_SIZE = 1
MAX_TITLE_SIZE = 32
MAX_QUESTION_SIZE = 128
MAX_ANSWER_SIZE = 128
BART_EMBEDDING_SIZE = 1024
MAX_TITLE_QUESTION_SIZE = MAX_TITLE_SIZE + MAX_QUESTION_SIZE
LABELS_COUNT = len(output_categories)

In [0]:
from fairseq.data.data_utils import collate_tokens

def extract_features_for_batch(batch_snt, max_length):
  batch_encoded = []
  trimmed_count = 0
  for s in batch_snt:
    encoded = bart.encode(s)
    if len(encoded) <= max_length:
      encoded = torch.cat((encoded, torch.tensor([1]).repeat(max_length - len(encoded))), 0)
    else:
      encoded = encoded[:max_length]
      trimmed_count += 1
    batch_encoded.append(encoded)
  batch = collate_tokens(batch_encoded, pad_idx=1)
  # print('trimmed_count:', trimmed_count)
  return batch

In [143]:
from tqdm import tqdm
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def prepare_dataset(inpput_df, batch_size, with_target=True):
  questions = []
  answers = []
  output = []
  data_q = torch.zeros([0, batch_size, MAX_TITLE_QUESTION_SIZE], dtype=torch.int64)
  data_a = torch.zeros([0, batch_size, MAX_ANSWER_SIZE], dtype=torch.int64)
  if with_target:
    target = torch.zeros([0, batch_size, LABELS_COUNT], dtype=torch.float32)
  for index, row in tqdm(inpput_df.iterrows()):
    questions.append(row['question_title'] + ' ' + row['question_body'])
    answers.append(row['answer'])
    if with_target:
      output.append(row[output_categories].values)
    if len(questions) < batch_size:
      continue
    
    questions_f = extract_features_for_batch(questions, MAX_TITLE_QUESTION_SIZE)
    answers_f = extract_features_for_batch(answers, MAX_ANSWER_SIZE)
    feat_q = questions_f.unsqueeze(0)
    feat_a = answers_f.unsqueeze(0)

    data_q = torch.cat((data_q, feat_q), 0)
    data_a = torch.cat((data_a, feat_a), 0)
    if with_target:
      output = torch.Tensor(output).unsqueeze(0)
      target = torch.cat((target, output), 0)

    questions = []
    answers = []
    output = []
  # TODO: extract the rest?

  data_q = data_q.contiguous().to(device)
  data_a = data_a.contiguous().to(device)
  if with_target:
    target = target.contiguous().to(device)
  print('data_q:', data_q.shape)
  print('data_a:', data_a.shape)
  if with_target:
    print('target:', target.shape)
  if with_target:
    return data_q, data_a, target
  else:
    return data_q, data_a

train_q, train_a, train_target = prepare_dataset(df_train, TRAIN_BATCH_SIZE)
test_q, test_a = prepare_dataset(df_test, EVAL_BATCH_SIZE, False)

6079it [00:42, 143.13it/s]
11it [00:00, 102.14it/s]

data_q: torch.Size([759, 8, 160])
data_a: torch.Size([759, 8, 128])
target: torch.Size([759, 8, 30])


476it [00:03, 157.22it/s]

data_q: torch.Size([476, 1, 160])
data_a: torch.Size([476, 1, 128])


## Build pytorch model

In [0]:
import math
import torch
import torch.nn.functional as F
import torch.nn as nn

class BARTClassificationHead(nn.Module):
    """Head for sentence-level classification tasks."""

    def __init__(
        self,
        input_dim,
        inner_dim,
        num_classes,
        activation_fn,
        pooler_dropout,
    ):
        super().__init__()
        self.dense = nn.Linear(input_dim, inner_dim)
        self.activation_fn = activation_fn
        self.dropout = nn.Dropout(p=pooler_dropout)
        self.out_proj = nn.Linear(inner_dim, num_classes)

    def forward(self, features, **kwargs):
        x = features
        x = self.dropout(x)
        # print('dense', x.shape)
        x = self.dense(x)
        # print('activation_fn', x.shape)
        # print('x0:', x.cpu())
        x = self.activation_fn(x)
        # print('dropout', x.shape)
        # print('x1:', x.cpu())
        x = self.dropout(x)
        # print('out_proj', x.shape)
        x = self.out_proj(x)
        # print('return', x.shape)
        return x

class BartMultiLabelModel(nn.Module):

    def __init__(self, bart, q_size, a_size, bart_embedding_size, labels_count, dropout):
        super(BartMultiLabelModel, self).__init__()
        
        self.bart = bart
        self.dense = nn.Linear(q_size + a_size, 1)
        inner_dimension = 512
        self.head = BARTClassificationHead(
            bart_embedding_size, inner_dimension,
            labels_count, torch.tanh, dropout
            )

    def forward(self, src_q, src_a):
        # print('src_q:', src_q.cpu().numpy())
        # print('src_a:', src_a.cpu().numpy())
        features_q = self.bart.extract_features(src_q)
        features_a = self.bart.extract_features(src_a)
        # print('features_q:', features_q.cpu())
        # print('features_a:', features_a.cpu())
        sys.stdout.flush()
        features = torch.cat((features_q, features_a), 1)
        # print('permute:', features.shape)
        features = features.permute(0, 2, 1)
        # print('dense:', features.shape)
        # print('features0:', features.cpu())
        features = self.dense(features).squeeze(2)
        # print('head:', features.shape)
        # print('features1:', features.cpu())
        labels_logits = self.head(features)
        # print('labels_logits:', labels_logits.cpu())
        labels = torch.sigmoid(labels_logits)
        return labels

## Initiate an instance

In [0]:
dropout = 0.2 # the dropout value
model = BartMultiLabelModel(bart, MAX_TITLE_QUESTION_SIZE, MAX_ANSWER_SIZE, BART_EMBEDDING_SIZE, LABELS_COUNT, dropout).to(device)

In [146]:
torch.cuda.memory_allocated() / 1024 / 1024 / 1024

3.127530097961426

## Run the model

In [0]:
import numpy as np
from scipy.stats import spearmanr

def compute_spearmanr_ignore_nan(trues, preds):
    rhos = []
    for tcol, pcol in zip(np.transpose(trues), np.transpose(preds)):
        # print('tcol:', tcol, '\npcol:', pcol)
        rhos.append(spearmanr(tcol, pcol).correlation)
    return np.nanmean(rhos)

In [0]:
import sys
criterion = nn.BCELoss()
lr = 2e-5 # learning rate
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)

import time
def train(train_q, train_a, train_target):
    model.train() # Turn on the train mode
    total_loss = 0.
    start_time = time.time()
    for i in range(0, train_q.size(0)):
        question, answer, target = train_q[i], train_a[i], train_target[i]
        optimizer.zero_grad()
        print(f'step {i}')
        # print(question.cpu())
        # print(answer.cpu())
        output = model(question, answer)
        # print(target.cpu())
        # print(output.cpu())
        assert len(((target > 1.0) | (target < 0.0)).nonzero().cpu().numpy()) == 0
        assert len(((output > 1.0) | (output < 0.0)).nonzero().cpu().numpy()) == 0
        loss = criterion(output, target)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()
        sys.stdout.flush()

        total_loss += loss.item()
        log_interval = 50
        if i % log_interval == 0 and i > 0:
            cur_loss = total_loss / log_interval
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches | '
                  'lr {:02.6f} | ms/batch {:5.2f} | '
                  'loss {:5.2f} | ppl {:8.2f}'.format(
                    epoch, i, len(train_q), scheduler.get_lr()[0],
                    elapsed * 1000 / log_interval,
                    cur_loss, math.exp(cur_loss)))
            total_loss = 0
            start_time = time.time()

dbg_output = None
dbg_target = None

test_result = []

def evaluate(eval_model, valid_q, valid_a, valid_target, save_test):
    global dbg_output
    global dbg_target
    global test_result

    output_all = []
    target_all = []
    test_all = []
    eval_model.eval() # Turn on the evaluation mode
    with torch.no_grad():
        for i in range(0, valid_q.size(0)):
            question, answer, target = valid_q[i], valid_a[i], valid_target[i]
            output = eval_model(question, answer)
            output_all += output.cpu().numpy().tolist()
            target_all += target.cpu().numpy().tolist()

        dbg_output = output_all
        dbg_target = target_all
        total_loss = compute_spearmanr_ignore_nan(output_all, target_all)

        if save_test:
          for i in range(0, test_q.size(0)):
              question, answer = test_q[i], test_a[i]
              output = eval_model(question, answer)
              test_all += output.cpu().numpy().tolist()
          test_result.append(test_all)

    return total_loss



Loop over epochs. Save the model if the validation loss is the best
we've seen so far. Adjust the learning rate after each epoch.



In [0]:
from sklearn.model_selection import GroupKFold
train_indexes = list(range(train_q.shape[0]))
gkf = GroupKFold(n_splits=5).split(X=train_indexes, groups=train_indexes)

best_val_loss = 0
epochs = 3 # The number of epochs
best_model = None

valid_preds = []
test_preds = []
for fold, (train_idx, valid_idx) in enumerate(gkf):

  # if fold in [0, 2]:
  print(f'fold: {fold}')
  train_input_q, train_input_a, train_input_t = train_q[train_idx], train_a[train_idx], train_target[train_idx]
  valid_input_q, valid_input_a, valid_input_t = train_q[valid_idx], train_a[valid_idx], train_target[valid_idx]

  for epoch in range(1, epochs + 1):
      epoch_start_time = time.time()
      train(train_input_q, train_input_a, train_input_t)
      val_loss = evaluate(model, valid_input_q, valid_input_a, valid_input_t, epoch == 3)
      print('-' * 89)
      print('| end of epoch {:3d} | time: {:5.2f}s | valid loss {:5.2f} | '
            'valid ppl {:8.2f}'.format(epoch, (time.time() - epoch_start_time),
                                      val_loss, math.exp(val_loss)))
      print('-' * 89)

      if val_loss > best_val_loss:
          best_val_loss = val_loss
          best_model = model

      scheduler.step()

  torch.save(model.state_dict(), OUTPUT_PATH + f'fold_{fold}.cpt')

fold: 0
step 0
step 1
step 2
step 3
step 4
step 5
step 6
step 7
step 8
step 9
step 10
step 11
step 12
step 13
step 14
step 15
step 16
step 17
step 18
step 19
step 20
step 21
step 22
step 23
step 24
step 25
step 26
step 27
step 28
step 29
step 30
step 31
step 32
step 33
step 34
step 35
step 36
step 37
step 38
step 39
step 40
step 41
step 42
step 43
step 44
step 45
step 46
step 47
step 48
step 49
step 50
| epoch   1 |    50/  607 batches | lr 0.000020 | ms/batch 586.92 | loss  0.69 | ppl     2.00
step 51
step 52
step 53
step 54
step 55
step 56
step 57
step 58
step 59
step 60
step 61
step 62
step 63
step 64
step 65
step 66
step 67
step 68
step 69
step 70
step 71
step 72
step 73
step 74
step 75
step 76
step 77
step 78
step 79
step 80
step 81
step 82
step 83
step 84
step 85
step 86
step 87
step 88
step 89
step 90
step 91
step 92
step 93
step 94
step 95
step 96
step 97
step 98
step 99
step 100
| epoch   1 |   100/  607 batches | lr 0.000020 | ms/batch 596.71 | loss  0.62 | ppl     1.87
step 

/usr/local/lib/python3.6/dist-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/usr/local/lib/python3.6/dist-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:1912: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


-----------------------------------------------------------------------------------------
| end of epoch   1 | time: 438.09s | valid loss  0.30 | valid ppl     1.35
-----------------------------------------------------------------------------------------
step 0
step 1
step 2
step 3
step 4
step 5
step 6
step 7
step 8
step 9
step 10
step 11
step 12
step 13
step 14
step 15
step 16
step 17
step 18
step 19
step 20
step 21
step 22
step 23
step 24
step 25
step 26
step 27
step 28
step 29
step 30
step 31
step 32
step 33
step 34
step 35
step 36
step 37
step 38
step 39
step 40
step 41
step 42
step 43
step 44
step 45
step 46
step 47
step 48
step 49
step 50
| epoch   2 |    50/  607 batches | lr 0.000015 | ms/batch 588.26 | loss  0.41 | ppl     1.51
step 51
step 52
step 53
step 54
step 55
step 56
step 57
step 58
step 59
step 60
step 61
step 62
step 63
step 64
step 65
step 66
step 67
step 68
step 69
step 70
step 71
step 72
step 73
step 74
step 75
step 76
step 77
step 78
step 79
step 80
step 81
step 

In [0]:
df_sub.iloc[:, 1:] = np.average(test_result, axis=0) # for weighted average set weights=[...]

df_sub.to_csv('submission.csv', index=False)

In [0]:
df_sub